<a href="https://colab.research.google.com/github/zeinabmohamed/CIT690E-DeepLearning-Zeinab_Abdelmawla-191009/blob/main/chatbot/eval_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
def readJson(filename):
  with open(filename, 'r') as file:
    intents = json.load(file)
    return intents

In [4]:
questionDoctorQAs = readJson('questionDoctorQAs.json')

In [5]:
import pandas as pd
all_medical_df = pd.DataFrame.from_dict(questionDoctorQAs)
all_medical_df

,answer,answer_author,question,question_text,tags,url
0,you can stay assured you are fine free of hiv....,Dr. Ayman Darrag,is my anti hiv test conclusive or need retest?,Is my Anti Hiv Test Conclusive or need retest?,[hiv test],https://questiondoctors.com/is-my-anti-hiv-tes...
1,hi you are 100 % hiv free good luck,Dr Ahmed Fawzy,is my anti hiv test conclusive or need retest?,Is my Anti Hiv Test Conclusive or need retest?,[hiv test],https://questiondoctors.com/is-my-anti-hiv-tes...
2,your time slots of the tests and the final res...,Dr.Honey,is my anti hiv test conclusive or need retest?,Is my Anti Hiv Test Conclusive or need retest?,[hiv test],https://questiondoctors.com/is-my-anti-hiv-tes...
3,hi i see no labrum tear however i see acetabul...,Dr Ahmed Fawzy,i have some hip pain 9 weeks. had mra image re...,I have some hip pain 9 weeks. Had MRA image re...,[magnetic resonance angiography (mra)],https://questiondoctors.com/i-have-some-hip-pa...
4,no visible tear in labrum but obviously calcif...,Dr. Ayman Darrag,i have some hip pain 9 weeks. had mra image re...,I have some hip pain 9 weeks. Had MRA image re...,[magnetic resonance angiography (mra)],https://questiondoctors.com/i-have-some-hip-pa...
...,...,...,...,...,...,...
5674,get it confirmed by doing venous doppler of bo...,Dr Mahaveer,pain in legs – varicose veins?,Question: Pain in Legs – Varicose veins?,[],https://questiondoctors.com/question-pain-in-l...
5675,hi dear. there are very less chances of pregna...,Dr Mahaveer,headaches really tired all the time feeling na...,"Question: Headaches, really tired all the time...",[don't know if its too early to take a pregnan...,https://questiondoctors.com/question-headaches...
5676,i’m a radiologist. may be an mri misread. when...,Dr. Erik Ramonov,46 year old male had stroke on right side of b...,Question: 46 year old male had stroke on right...,[stroke on right side of brain],https://questiondoctors.com/question-46-year-o...
5677,could be reaction to oxycodone…. ”the most fre...,Jason Roberge,46 year old male had stroke on right side of b...,Question: 46 year old male had stroke on right...,[stroke on right side of brain],https://questiondoctors.com/question-46-year-o...


In [6]:
from gensim.models import KeyedVectors
!pip install wget
import wget

url = 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
filename = wget.download(url)

embedding_path = 'GoogleNews-vectors-negative300.bin.gz'
embedding_dict_lib = KeyedVectors.load_word2vec_format(embedding_path, binary=True)

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=ac88fdac0433bc50c289dd07fede25d1bca0cb371052b2b44d0daf6319723bf7
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [15]:
import tensorflow as tf

model = tf.keras.models.load_model('./en_ameseLSTM.h5', custom_objects={'ManDist': ManDist})
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 32, 300)      3516600     ['input_3[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, 32, 300)      3516600     ['input_4[0][0]']                
                                                                                            

In [ ]:
question_col = all_medical_df["question"]
final_eval_result = pd.DataFrame(columns=['InputQ','RealatedQ','RealatedQ_prediaction','RealatedQ_sucess_predication','RealatedQ_index','RealatedQ_question_text','RealatedQ_answer','RealatedQ_answer_author','RealatedQ_tags','RealatedQ_url'])
#range(len(question_col))
for curentSelectedQIndex in range(len(question_col)):
  # preaper test DF
  test_df = pd.DataFrame(question_col)
  test_df.columns = ['question1']
  test_df['question2']=test_df['question1'][curentSelectedQIndex]
  ##################

  predictions = predirct(test_df,embedding_dict_lib)

  Eval = pd.DataFrame(columns= ['question1','question2','prediction'])
  Eval['question1']= test_df['question1']
  Eval['question2']=test_df['question2']
  Eval['prediction']= predictions
  
  ######### start predicat related Question ############

  validateEvalDirExist('./Eval/')
  evalFileName = './Eval/Eval_question_'+str(curentSelectedQIndex)+'.csv'
  Eval.to_csv(evalFileName)
  column = Eval["prediction"]
  predictionـcol = Eval["prediction"]
  max_index = predictionـcol.idxmax()
  max_predited_quetion_row = Eval.iloc[[max_index]]
  prediacted_related_question_row = all_medical_df.iloc[[max_index]]
  print("Question .. ",curentSelectedQIndex," : ",max_predited_quetion_row.iloc[0]['question2'])

  ######### save predicated Question ############
  isSucess = 1 if prediacted_related_question_row.iloc[0]['question']== max_predited_quetion_row.iloc[0]['question2'] else 0
  new_row = {'InputQ':max_predited_quetion_row.iloc[0]['question2'],
             'RealatedQ':prediacted_related_question_row.iloc[0]['question'],
             'RealatedQ_prediaction':max_predited_quetion_row.iloc[0]['prediction'],
             'RealatedQ_sucess_predication':isSucess,
             'RealatedQ_index':max_index,
             'RealatedQ_question_text':prediacted_related_question_row.iloc[0]['question_text'],
             'RealatedQ_answer':prediacted_related_question_row.iloc[0]['answer'],
             'RealatedQ_answer_author':prediacted_related_question_row.iloc[0]['answer_author'],
             'RealatedQ_tags':prediacted_related_question_row.iloc[0]['tags'],
             'RealatedQ_url':prediacted_related_question_row.iloc[0]['url']
             }
  final_eval_result = final_eval_result.append(new_row,ignore_index=True)

final_eval_result.head(10)

preicatedSucessRate = (final_eval_result['RealatedQ_sucess_predication'].sum()/len(final_eval_result))*100
print("preicatedSucessRate",preicatedSucessRate)
######### Export final result to csv ######### 
finalEvalFileName = './Eval/Final_Eval_question.csv'
final_eval_result.to_csv(finalEvalFileName)

1000 sentences embedded.
2000 sentences embedded.
3000 sentences embedded.
4000 sentences embedded.
5000 sentences embedded.
Question ..  0  :  is my anti hiv test conclusive or need retest?
1000 sentences embedded.
2000 sentences embedded.
3000 sentences embedded.
4000 sentences embedded.
5000 sentences embedded.
Question ..  1  :  is my anti hiv test conclusive or need retest?
1000 sentences embedded.
2000 sentences embedded.
3000 sentences embedded.
4000 sentences embedded.
5000 sentences embedded.
Question ..  2  :  is my anti hiv test conclusive or need retest?
1000 sentences embedded.
2000 sentences embedded.
3000 sentences embedded.
4000 sentences embedded.
5000 sentences embedded.
Question ..  3  :  i have some hip pain 9 weeks. had mra image recently would welcome second opinion?
1000 sentences embedded.
2000 sentences embedded.
3000 sentences embedded.
4000 sentences embedded.
5000 sentences embedded.
Question ..  4  :  i have some hip pain 9 weeks. had mra image recently wou

## Helper Functaions:

In [9]:
import pandas as pd

def predirct(input_test_df,embedding_dict):
  # Load training set
  for q in ['question1', 'question2']:
    input_test_df[q + '_n'] = input_test_df[q]

  # Make word2vec embeddings
  embedding_dim = 300
  max_seq_length = 32
  
  test_df, embeddings = make_w2v_embeddings(embedding_dict, input_test_df, embedding_dim=embedding_dim)

  # Split to dicts and append zero padding.
  X_test = split_and_zero_padding(test_df, max_seq_length)

  # Make sure everything is ok
  assert X_test['left'].shape == X_test['right'].shape

  # --

  prediction = model.predict([X_test['left'], X_test['right']])
  return prediction

In [10]:
def text_to_word_list(text):  # 文本分词
    text = str(text)
    text = text.lower()
    # Text cleaning rules for English text
    import re
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

In [11]:
import numpy as np

def make_w2v_embeddings(word2vec, df, embedding_dim):  # 将词转化为词向量
    vocabs = {}  # 词序号
    vocabs_count = 0  # 词个数计数器

    vocabs_not_w2v = {}  # 无法用词向量表示的词
    vocabs_not_w2v_count = 0  # Word count that cannot be represented by word vectors

    # 停用词
    # stops = set(open('data/stopwords.txt').read().strip().split('\n'))

    for index, row in df.iterrows():
        # 打印处理进度
        if index != 0 and index % 1000 == 0:
            print(str(index) + " sentences embedded.")

        for question in ['question1', 'question2']:
            q2n = []  # q2n -> question to numbers representation
            words = text_to_word_list(row[question])

            for word in words:
                # if word in stops:  # remove stop words
                # continue
                # The word of OOV is put into a dictionary that cannot be represented by a word vector, and the value is 1
                if word not in word2vec and word not in vocabs_not_w2v:  
                    vocabs_not_w2v_count += 1
                    vocabs_not_w2v[word] = 1   
                if word not in vocabs:  # Non-OOV words, extract the corresponding id
                    vocabs_count += 1
                    vocabs[word] = vocabs_count
                    q2n.append(vocabs_count)
                else:
                    q2n.append(vocabs[word])
            df.at[index, question + '_n'] = q2n

    embeddings = 1 * np.random.randn(len(vocabs) + 1, embedding_dim)  # 随机初始化一个形状为[全部词个数，词向量维度]的矩阵
    '''
    词1 [a1, a2, a3, ..., a60]
    词2 [b1, b2, b3, ..., b60]
    词3 [c1, c2, c3, ..., c60]
    '''
    embeddings[0] = 0  # 第一行用0填充，因为不存在index为0的词

    for index in vocabs:
        vocab_word = vocabs[index]
        if vocab_word in word2vec:
            embeddings[index] = word2vec[vocab_word]
    del word2vec

    return df, embeddings

In [12]:
from keras.preprocessing.sequence import pad_sequences
import itertools

def split_and_zero_padding(df, max_seq_length):  # 调整tokens长度

    # 训练集矩阵转换成字典
    X = {'left': df['question1_n'], 'right': df['question2_n']}

    # 调整到规定长度
    for dataset, side in itertools.product([X], ['left', 'right']):
        dataset[side] = pad_sequences(dataset[side], padding='pre', truncating='post', maxlen=max_seq_length)

    return dataset

In [13]:
from keras.layers import Layer
from keras import backend as K

class ManDist(Layer):  # 封装成keras层的曼哈顿距离计算

    # 初始化ManDist层，此时不需要任何参数输入
    def __init__(self, **kwargs):
        self.result = None
        super(ManDist, self).__init__(**kwargs)

    # Automatically build ManDist layer
    def build(self, input_shape):
        super(ManDist, self).build(input_shape)

    # Calculate Manhattan distance
    def call(self, x, **kwargs):
        self.result = K.exp(-K.sum(K.abs(x[0] - x[1]), axis=1, keepdims=True))
        return self.result

    # return result
    def compute_output_shape(self, input_shape):
        return K.int_shape(self.result)

In [14]:
import os
def validateEvalDirExist(dirpath):
  dirname = os.path.dirname(dirpath)
  if not os.path.exists(dirname):
    os.makedirs(dirname)